In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import pprint
import pylab as pl
import numpy as np


 

# Limit rows disp# Limit rows displayed in notebook
pd.set_option('display.max_rows', 20)
pd.set_option('display.precision', 2)

In [2]:
import json
import pandas
 
def convertBusiness(x):
    ob = json.loads(x)
    return {"business_id":ob["business_id"], "stars":ob["stars"], "city":ob["city"]}

def convertReviews(x):
    ''' Convert a json string to a flat python dictionary
    which can be passed into Pandas. '''
    ob = json.loads(x)
    return {"business_id":ob["business_id"], "stars":ob["stars"], "date":ob["date"]}

def createPandas(func, json_filename):
    result=[]
    f=open(json_filename, 'r')
    for line in f:
        result.append(func(line))
    return pd.DataFrame(result)

print "Creating Business Pandas"
pd_business = createPandas(convertBusiness,'yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_business.json')
print "Creating Review Pandas"
pd_review = createPandas(convertReviews, 'yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_review.json')
print "Completed Creating Pandas"


Creating Business Pandas
Creating Review Pandas
Completed Creating Pandas


In [7]:
#preprocess the business data
pd_business_lasvegas = pd_business[pd_business["city"] == "Las Vegas"]
#remove the unwanted stars data present in business.json
pd_business_lasvegas.pop('stars')

pd_review_lasvegas = pd.merge(pd_business_lasvegas, pd_review, left_on='business_id', right_on='business_id', how='inner')
print list(pd_review_lasvegas.columns.values)
gb = pd_review_lasvegas.groupby(['business_id'],as_index=False)
#print gb.size()
gb = gb.agg({'date' : np.min})
gb["new"] = (gb["date"] >= "2011-01-01")
print gb


['business_id', 'city', 'date', 'stars']
                  business_id        date    new
0      --1emggGHgoG6ipd_RMb-g  2013-03-30   True
1      --Dl2rW_xO8GuYBomlg9zw  2010-05-04  False
2      --jFTZmywe7StuZ2hEjxyA  2012-10-24   True
3      --sSW-WY3vyASh_eVPGUAw  2010-09-09  False
4      --zgHBiQpr8H2ZqSdGmguQ  2009-04-10  False
5      -0Ackw6MF82PXO9f9Jh_Kg  2009-05-24  False
6      -0VK5Z1BfUHUYq4PoBYNLw  2013-11-08   True
7      -0lRv242OzyPUB8c5E_UhQ  2011-06-07   True
8      -1B-DEGkLE1kDj5ENAF2NQ  2011-06-23   True
9      -1ERbsOf9XOC9wGbZYFr7g  2013-11-02   True
...                       ...         ...    ...
13576  zwCeWkw1emDT9RtBjAlt5A  2011-08-25   True
13577  zxD8WoHXyXDCD_sTywmogQ  2007-05-22  False
13578  zxNT4025QCHXNf10WJYZkw  2014-11-02   True
13579  zxo9nZwhlW1ZM7OUJP_1ag  2011-07-25   True
13580  zxviVY0ypMZMkZDfE0Ffqw  2012-09-11   True
13581  zyD03W9T-raaNmoEdodFQA  2014-06-08   True
13582  zyFZme7L_6kfvex2sgPfCA  2010-12-23  False
13583  zypmytGA92yk45AYn60ZH

                   business_id       city  stars_x        date  stars_y
0       uzztTCsg3XAKlYqmLO40aw  Las Vegas      4.0  2010-11-19        4
1       uzztTCsg3XAKlYqmLO40aw  Las Vegas      4.0  2011-09-11        4
2       uzztTCsg3XAKlYqmLO40aw  Las Vegas      4.0  2012-03-27        5
3       uzztTCsg3XAKlYqmLO40aw  Las Vegas      4.0  2012-07-25        3
4       uzztTCsg3XAKlYqmLO40aw  Las Vegas      4.0  2013-11-21        5
5       uzztTCsg3XAKlYqmLO40aw  Las Vegas      4.0  2014-01-19        5
6       uzztTCsg3XAKlYqmLO40aw  Las Vegas      4.0  2014-10-22        5
7       uzztTCsg3XAKlYqmLO40aw  Las Vegas      4.0  2014-12-16        2
8       IspFZDcpu9W5NvvJ7QgJZg  Las Vegas      4.5  2010-11-06        5
9       IspFZDcpu9W5NvvJ7QgJZg  Las Vegas      4.5  2010-12-29        5
...                        ...        ...      ...         ...      ...
617348  6TPxhpHqFedjMvBuw6pF3w  Las Vegas      4.5  2014-01-15        5
617349  6TPxhpHqFedjMvBuw6pF3w  Las Vegas      4.5  2014-03-26  

In [8]:
pd_lasvegas = pd.merge(pd_review_lasvegas, gb, left_on='business_id', right_on='business_id', how='inner')
pd_lasvegas.rename(columns={'date_y': 'start_date'}, inplace=True)
print pd_lasvegas


                   business_id       city      date_x  stars  start_date  \
0       uzztTCsg3XAKlYqmLO40aw  Las Vegas  2010-11-19      4  2010-11-19   
1       uzztTCsg3XAKlYqmLO40aw  Las Vegas  2011-09-11      4  2010-11-19   
2       uzztTCsg3XAKlYqmLO40aw  Las Vegas  2012-03-27      5  2010-11-19   
3       uzztTCsg3XAKlYqmLO40aw  Las Vegas  2012-07-25      3  2010-11-19   
4       uzztTCsg3XAKlYqmLO40aw  Las Vegas  2013-11-21      5  2010-11-19   
5       uzztTCsg3XAKlYqmLO40aw  Las Vegas  2014-01-19      5  2010-11-19   
6       uzztTCsg3XAKlYqmLO40aw  Las Vegas  2014-10-22      5  2010-11-19   
7       uzztTCsg3XAKlYqmLO40aw  Las Vegas  2014-12-16      2  2010-11-19   
8       IspFZDcpu9W5NvvJ7QgJZg  Las Vegas  2010-11-06      5  2010-11-06   
9       IspFZDcpu9W5NvvJ7QgJZg  Las Vegas  2010-12-29      5  2010-11-06   
...                        ...        ...         ...    ...         ...   
617348  6TPxhpHqFedjMvBuw6pF3w  Las Vegas  2014-01-15      5  2012-10-02   
617349  6TPx

In [9]:
pd_old_business_vegas = pd_lasvegas[pd_lasvegas['new'] == False]
pd_new_business_vegas = pd_lasvegas[pd_lasvegas['new'] == True]
print pd_old_business_vegas
print pd_new_business_vegas

                   business_id       city      date_x  stars  start_date  \
0       uzztTCsg3XAKlYqmLO40aw  Las Vegas  2010-11-19      4  2010-11-19   
1       uzztTCsg3XAKlYqmLO40aw  Las Vegas  2011-09-11      4  2010-11-19   
2       uzztTCsg3XAKlYqmLO40aw  Las Vegas  2012-03-27      5  2010-11-19   
3       uzztTCsg3XAKlYqmLO40aw  Las Vegas  2012-07-25      3  2010-11-19   
4       uzztTCsg3XAKlYqmLO40aw  Las Vegas  2013-11-21      5  2010-11-19   
5       uzztTCsg3XAKlYqmLO40aw  Las Vegas  2014-01-19      5  2010-11-19   
6       uzztTCsg3XAKlYqmLO40aw  Las Vegas  2014-10-22      5  2010-11-19   
7       uzztTCsg3XAKlYqmLO40aw  Las Vegas  2014-12-16      2  2010-11-19   
8       IspFZDcpu9W5NvvJ7QgJZg  Las Vegas  2010-11-06      5  2010-11-06   
9       IspFZDcpu9W5NvvJ7QgJZg  Las Vegas  2010-12-29      5  2010-11-06   
...                        ...        ...         ...    ...         ...   
615712  -G4Hvq3gUYJRQhGTGRXbCw  Las Vegas  2014-12-08      1  2009-11-17   
615713  -G4H

In [ ]:
#calculate the mean change in the ratings for all old business given a start date of one new business
def find_mean_change(start_date):


    pd_old_business_vegas['before']=(pd_old_business_vegas['date_x'] < start_date)
    pd_old_before=pd_old_business_vegas[pd_old_business_vegas['before']==True]
    pd_old_after=pd_old_business_vegas[pd_old_business_vegas['before']==False]
    #print pd_old_after

    gb_before = pd_old_before.groupby(['business_id'], as_index=False)
    gb_before = gb_before.agg({'stars' : np.mean})

    gb_after=pd_old_after.groupby(['business_id'], as_index=False)
    gb_after=gb_after.agg({'stars' : np.mean})

    before_after_ratings=pd.merge(gb_before, gb_after, left_on='business_id', right_on='business_id', how='inner')
    before_after_ratings.rename(columns={'stars_x': 'before_rating'}, inplace=True)
    before_after_ratings.rename(columns={'stars_y': 'after_rating'}, inplace=True)
    before_after_ratings['diff']=before_after_ratings['after_rating']-before_after_ratings['before_rating']
    #print before_after_ratings

    mean=before_after_ratings[['diff']].mean()
    print mean
#calculate mean for every new business
start_date= pd_new_business_vegas['start_date'].iloc[0]
find_mean_change(start_date)

for index, row in pd_new_business_vegas.iterrows():
    find_mean_change(row['start_date'])

diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
diff   -0.1
dtype: float64
d